In [1]:
# Importing Dependencies
import numpy as np
import gym
from gym.envs.box2d.car_racing import *
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization
from keras.optimizers import Adam
from statistics import mean
from pyglet.window import key
import time
from tqdm import tqdm 
import os
from keras.utils import normalize

C:\Users\Piyush\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Function for generating data by playing
def start_playing(n_episodes):
    a = np.array( [0.0, 0.0, 0.0] )
    def key_press(k, mod):
        global restart
        if k==0xff0d: restart = True
        if k==key.LEFT:  a[0] = -1.0
        if k==key.RIGHT: a[0] = +1.0
        if k==key.UP:    a[1] = +1.0
        if k==key.DOWN:  a[2] = +0.8   
    def key_release(k, mod):
        if k==key.LEFT  and a[0]==-1.0: a[0] = 0
        if k==key.RIGHT and a[0]==+1.0: a[0] = 0
        if k==key.UP:    a[1] = 0
        if k==key.DOWN:  a[2] = 0
    
    env = CarRacing()
    env.render()
    env.viewer.window.on_key_press = key_press
    env.viewer.window.on_key_release = key_release
    
    for episode in range(n_episodes):
        prev_observation = env.reset()
        if episode==0:
            X = np.array(prev_observation.tolist())
            y = np.array([0.,0.,0.])
        restart = False
        while True:
            action = a
            observation, reward, done, info = env.step(action)
            X = np.vstack((X, prev_observation))
            y = np.vstack((y, action))
            prev_observation=observation
            env.render()
            if done or restart: break
        time.sleep(2)    
    np.save('train_X.npy', X)
    np.save('train_y.npy', y)    
    env.close()
    return X, y

In [3]:
# Playing the to create data
X, y=start_playing(20)

Track generation: 1121..1405 -> 284-tiles track
Track generation: 925..1162 -> 237-tiles track
retry to generate track (normal if there are not many of this messages)
Track generation: 1228..1538 -> 310-tiles track
Track generation: 1132..1419 -> 287-tiles track
Track generation: 1282..1607 -> 325-tiles track
Track generation: 1171..1472 -> 301-tiles track
retry to generate track (normal if there are not many of this messages)
Track generation: 1120..1404 -> 284-tiles track
Track generation: 1030..1291 -> 261-tiles track
Track generation: 1119..1403 -> 284-tiles track
Track generation: 1126..1412 -> 286-tiles track
Track generation: 1413..1770 -> 357-tiles track
Track generation: 1230..1541 -> 311-tiles track
Track generation: 1246..1562 -> 316-tiles track
Track generation: 1128..1414 -> 286-tiles track
Track generation: 1085..1365 -> 280-tiles track
Track generation: 1387..1738 -> 351-tiles track
Track generation: 992..1251 -> 259-tiles track
Track generation: 1029..1296 -> 267-tiles 

In [8]:
# Combining gameplay files
temp = np.load('train_data-549.4999999999591.npy')
tempX = [i[0] for i in temp]
tempy = [i[1] for i in temp]
X = np.array(tempX)
y = np.array(tempy)
for file in tqdm(os.listdir('Data/')):
    temp = np.load('Data/' + file)
    tempX = [i[0] for i in temp]
    tempy = [i[1] for i in temp]
    tempX = np.array(tempX)
    tempy = np.array(tempy)
    X  = np.concatenate([X, tempX], axis=0)
    y  = np.concatenate([y, tempy], axis=0)

np.save('X.npy', X)    

100%|████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:05<00:00,  1.60it/s]


In [5]:
# Converting output to proper OneHot array
Y=[]
for i in range(len(y)):  
    if np.array_equal(y[i], np.array([0,1,0])):
        Y.append(np.array([0,0,1,0,0]))
    elif np.array_equal(y[i], np.array([0,0,0.8])):
        Y.append(np.array([0,0,0,1,0]))
    elif np.array_equal(y[i], np.array([1,0,0])):
        Y.append(np.array([0,1,0,0,0]))
    elif np.array_equal(y[i], np.array([-1,0,0])):
        Y.append(np.array([1,0,0,0,0]))
    else:
        Y.append(np.array([0,0,0,0,1]))      
      
np.save('y_modified.npy', Y)    

In [47]:
# Loading the data
X = np.load('X_balanced.npy')
y = np.load('y_balanced.npy')
X = X.reshape(-1,96,96,3)
y = y.reshape(-1,5)
print(X.shape)
print(y.shape)

(12192, 96, 96, 3)
(12192, 5)


In [49]:
# Variation in generated data
left=np.array([1,0,0,0])
right=np.array([0,1,0,0])
straight=np.array([0,0,1,0])
stop=np.array([0,0,0,1])
l=0
r=0
s=0
b=0
nk=0

for key in y:
    if np.array_equal(key, left):
        l+=1
    elif np.array_equal(key, right):
        r+=1
    elif np.array_equal(key, straight):
        s+=1
    elif np.array_equal(key, stop):
        b+=1
       
      
print(l,r,s,b)    

2694 1318 3001 179


In [48]:
new_X = np.array(X[0])
new_y = np.array([0,0,1,0])

moves={'l':0, 'r':0, 's':0, 'b':0}

# y_balanced
y = np.delete(y, 4, axis=1)

for ix, iy in tqdm(zip(X, y)):
    if np.array_equal(iy, left) and moves['l']<3000:
        new_X = np.vstack((new_X, ix))
        new_y = np.vstack((new_y, iy))
        moves['l'] += 1
    elif np.array_equal(iy, right) and moves['r']<3000:
        new_X = np.vstack((new_X, ix))
        new_y = np.vstack((new_y, iy))
        moves['r'] += 1
    elif np.array_equal(iy, straight) and moves['s']<3000:
        new_X = np.vstack((new_X, ix))
        new_y = np.vstack((new_y, iy))
        moves['s'] += 1
    elif np.array_equal(iy, stop) and moves['b']<3000:
        new_X = np.vstack((new_X, ix))
        new_y = np.vstack((new_y, iy)) 
        moves['b'] += 1
   
X = new_X.reshape(-1,96,96,3)
y = new_y

np.save('X_balanced4', X)
np.save('y_balanced4', y)


0it [00:00, ?it/s]
242it [00:00, 2370.42it/s]
364it [00:00, 1846.95it/s]
451it [00:00, 1137.07it/s]
530it [00:00, 678.14it/s] 
596it [00:00, 595.85it/s]
656it [00:00, 531.79it/s]
711it [00:01, 474.40it/s]
761it [00:01, 447.98it/s]
808it [00:01, 268.21it/s]
845it [00:01, 259.09it/s]
878it [00:01, 222.06it/s]
907it [00:01, 207.05it/s]
932it [00:02, 216.55it/s]
963it [00:02, 234.78it/s]
990it [00:02, 213.03it/s]
1030it [00:02, 241.89it/s]
1079it [00:02, 284.67it/s]
1113it [00:02, 198.50it/s]
1140it [00:03, 168.99it/s]
1163it [00:03, 155.59it/s]
1183it [00:03, 120.52it/s]
1200it [00:03, 89.18it/s] 
1214it [00:04, 75.69it/s]
1226it [00:04, 83.51it/s]
1237it [00:04, 87.83it/s]
1249it [00:04, 93.05it/s]
1260it [00:04, 89.88it/s]
1270it [00:04, 82.56it/s]
1292it [00:04, 99.12it/s]
1307it [00:04, 107.94it/s]
1321it [00:04, 115.02it/s]
1334it [00:05, 108.92it/s]
1346it [00:05, 107.19it/s]
1358it [00:05, 98.68it/s] 
1377it [00:05, 114.45it/s]
1390it [00:05, 109.11it/s]
1407it [00:05, 120.66it/s]

4130it [00:54, 26.25it/s]
4135it [00:54, 21.29it/s]
4139it [00:54, 18.68it/s]
4142it [00:54, 17.13it/s]
4145it [00:55, 16.29it/s]
4148it [00:55, 15.75it/s]
4155it [00:55, 19.99it/s]
4158it [00:55, 18.01it/s]
4161it [00:55, 16.78it/s]
4164it [00:56, 16.02it/s]
4166it [00:56, 15.45it/s]
4168it [00:56, 15.11it/s]
4170it [00:56, 14.88it/s]
4172it [00:56, 14.57it/s]
4213it [00:56, 20.37it/s]
4223it [00:57, 18.53it/s]
4231it [00:58, 17.10it/s]
4237it [00:58, 16.50it/s]
4242it [00:58, 16.10it/s]
4246it [00:59, 15.39it/s]
4249it [00:59, 15.34it/s]
4252it [00:59, 15.26it/s]
4255it [00:59, 15.13it/s]
4257it [00:59, 14.96it/s]
4259it [00:59, 14.88it/s]
4261it [01:00, 14.89it/s]
4263it [01:00, 14.86it/s]
4265it [01:00, 14.55it/s]
4285it [01:00, 19.95it/s]
4291it [01:00, 20.62it/s]
4296it [01:01, 18.47it/s]
4300it [01:01, 17.22it/s]
4303it [01:01, 16.43it/s]
4307it [01:01, 18.90it/s]
4310it [01:01, 17.41it/s]
4314it [01:02, 19.57it/s]
4317it [01:02, 20.25it/s]
4322it [01:02, 23.20it/s]
4327it [01:0

5537it [01:57, 38.57it/s]
5542it [01:58, 23.51it/s]
5546it [01:58, 19.15it/s]
5549it [01:58, 17.62it/s]
5554it [01:58, 21.77it/s]
5558it [01:58, 20.77it/s]
5561it [01:58, 22.42it/s]
5564it [01:59, 19.36it/s]
5567it [01:59, 14.75it/s]
5569it [01:59, 12.91it/s]
5571it [01:59, 14.17it/s]
5575it [01:59, 17.50it/s]
5587it [02:00, 22.29it/s]
5591it [02:00, 25.65it/s]
5595it [02:00, 20.05it/s]
5598it [02:00, 15.19it/s]
5601it [02:01, 12.93it/s]
5603it [02:01, 10.86it/s]
5605it [02:01,  9.51it/s]
5607it [02:01,  9.26it/s]
5609it [02:02,  8.92it/s]
5611it [02:02,  9.32it/s]
5613it [02:02,  9.64it/s]
5618it [02:02, 12.69it/s]
5621it [02:02, 11.51it/s]
5623it [02:03, 10.93it/s]
5625it [02:03, 10.43it/s]
5627it [02:03, 10.05it/s]
5629it [02:03,  9.97it/s]
5631it [02:03,  9.84it/s]
5633it [02:04,  9.79it/s]
5635it [02:04,  9.76it/s]
5637it [02:04,  9.93it/s]
5639it [02:04,  9.86it/s]
5641it [02:05,  9.77it/s]
5642it [02:05,  9.72it/s]
5643it [02:05,  9.69it/s]
5644it [02:05,  9.66it/s]
5645it [02:0

6375it [02:55, 13.00it/s]
6378it [02:55, 15.14it/s]
6381it [02:55, 12.08it/s]
6383it [02:56, 10.49it/s]
6387it [02:56, 13.17it/s]
6394it [02:56, 17.05it/s]
6398it [02:56, 16.91it/s]
6401it [02:56, 12.83it/s]
6404it [02:57, 10.98it/s]
6406it [02:57, 10.07it/s]
6408it [02:57,  9.42it/s]
6410it [02:58,  8.99it/s]
6412it [02:58,  8.73it/s]
6414it [02:58,  8.56it/s]
6415it [02:58,  8.40it/s]
6416it [02:58,  8.30it/s]
6417it [02:58,  8.22it/s]
6420it [02:59, 10.18it/s]
6423it [02:59, 12.26it/s]
6425it [02:59, 13.21it/s]
6427it [02:59, 13.93it/s]
6429it [02:59, 11.45it/s]
6431it [02:59, 12.54it/s]
6435it [02:59, 15.35it/s]
6437it [03:00, 12.05it/s]
6439it [03:00, 10.51it/s]
6441it [03:00,  9.59it/s]
6443it [03:00,  9.10it/s]
6445it [03:01,  8.67it/s]
6446it [03:01,  8.37it/s]
6453it [03:01, 11.24it/s]
6456it [03:01, 13.39it/s]
6459it [03:01, 13.00it/s]
6462it [03:02, 12.66it/s]
6464it [03:02, 13.45it/s]
6466it [03:02, 14.31it/s]
6470it [03:02, 17.13it/s]
6473it [03:02, 18.98it/s]
6476it [03:0

7271it [04:03,  6.99it/s]
7272it [04:04,  6.99it/s]
7273it [04:04,  7.08it/s]
7274it [04:04,  7.03it/s]
7275it [04:04,  7.00it/s]
7276it [04:04,  7.00it/s]
7277it [04:04,  7.00it/s]
7278it [04:04,  6.95it/s]
7279it [04:05,  6.95it/s]
7280it [04:05,  6.89it/s]
7281it [04:05,  6.92it/s]
7282it [04:05,  6.88it/s]
7283it [04:05,  6.90it/s]
7284it [04:05,  6.90it/s]
7285it [04:05,  7.04it/s]
7286it [04:06,  7.11it/s]
7287it [04:06,  7.03it/s]
7288it [04:06,  7.12it/s]
7289it [04:06,  7.05it/s]
7290it [04:06,  6.94it/s]
7291it [04:06,  6.93it/s]
7292it [04:06,  6.93it/s]
7293it [04:07,  6.92it/s]
7294it [04:07,  6.91it/s]
7295it [04:07,  6.90it/s]
7296it [04:07,  6.90it/s]
7297it [04:07,  6.88it/s]
7302it [04:07,  9.06it/s]
7304it [04:07, 10.10it/s]
7306it [04:08, 10.94it/s]
7309it [04:08, 12.73it/s]
7315it [04:08, 16.07it/s]
7318it [04:08, 17.22it/s]
7321it [04:08, 18.13it/s]
7324it [04:08, 18.79it/s]
7329it [04:08, 21.73it/s]
7332it [04:09, 16.33it/s]
7335it [04:09, 13.83it/s]
7337it [04:0

...

9693it [11:08,  5.00it/s]
9695it [11:08,  4.77it/s]
9696it [11:09,  4.45it/s]
9697it [11:09,  4.41it/s]
9698it [11:09,  4.38it/s]
9699it [11:09,  4.35it/s]
9700it [11:10,  4.33it/s]
9701it [11:10,  4.27it/s]
9702it [11:10,  4.27it/s]
9703it [11:10,  4.27it/s]
9704it [11:11,  4.26it/s]
9705it [11:11,  4.27it/s]
9706it [11:11,  4.27it/s]
9707it [11:11,  4.19it/s]
9708it [11:11,  4.21it/s]
9709it [11:12,  4.16it/s]
9710it [11:12,  4.20it/s]
9711it [11:12,  4.23it/s]
9712it [11:12,  4.22it/s]
9714it [11:13,  4.97it/s]
9715it [11:13,  4.66it/s]
9716it [11:13,  4.54it/s]
9717it [11:13,  4.46it/s]
9718it [11:14,  4.29it/s]
9719it [11:14,  4.19it/s]
9720it [11:14,  4.18it/s]
9721it [11:14,  4.16it/s]
9722it [11:15,  4.17it/s]
9723it [11:15,  4.21it/s]
9725it [11:15,  4.96it/s]
9726it [11:15,  4.73it/s]
9727it [11:16,  4.58it/s]
9728it [11:16,  4.48it/s]
9729it [11:16,  4.41it/s]
9730it [11:16,  4.36it/s]
9731it [11:16,  4.26it/s]
9732it [11:17,  4.19it/s]
9733it [11:17,  4.16it/s]
9734it [11:1

10254it [12:35,  4.10it/s]
10255it [12:35,  4.12it/s]
10256it [12:35,  4.10it/s]
10257it [12:36,  4.06it/s]
10258it [12:36,  4.02it/s]
10259it [12:36,  3.99it/s]
10260it [12:36,  3.97it/s]
10261it [12:37,  3.96it/s]
10262it [12:37,  3.94it/s]
10263it [12:37,  3.92it/s]
10264it [12:38,  3.91it/s]
10265it [12:38,  3.99it/s]
10266it [12:38,  3.97it/s]
10267it [12:38,  3.97it/s]
10274it [12:39,  5.33it/s]
10276it [12:39,  4.82it/s]
10302it [12:39,  6.74it/s]
10306it [12:40,  8.15it/s]
10314it [12:40, 10.48it/s]
10325it [12:40, 13.55it/s]
10329it [12:41,  7.81it/s]
10332it [12:42,  6.01it/s]
10334it [12:42,  5.18it/s]
10336it [12:43,  4.73it/s]
10338it [12:43,  4.44it/s]
10339it [12:44,  4.26it/s]
10341it [12:44,  4.93it/s]
10342it [12:44,  4.56it/s]
10343it [12:44,  4.35it/s]
10344it [12:45,  4.20it/s]
10345it [12:45,  4.12it/s]
10349it [12:45,  5.32it/s]
10350it [12:45,  4.80it/s]
10351it [12:46,  4.50it/s]
10352it [12:46,  4.31it/s]
10353it [12:46,  4.18it/s]
10354it [12:46,  4.02it/s]
1

10988it [14:09,  4.28it/s]
10990it [14:09,  4.17it/s]
10992it [14:10,  4.09it/s]
10993it [14:10,  4.05it/s]
10994it [14:10,  4.01it/s]
10995it [14:10,  3.96it/s]
11059it [14:11,  5.62it/s]
11067it [14:13,  4.93it/s]
11073it [14:14,  4.45it/s]
11077it [14:16,  4.22it/s]
11080it [14:16,  5.22it/s]
11083it [14:17,  4.71it/s]
11085it [14:17,  4.44it/s]
11087it [14:18,  4.28it/s]
11089it [14:18,  4.16it/s]
11090it [14:18,  4.06it/s]
11091it [14:19,  3.95it/s]
11092it [14:19,  3.92it/s]
11093it [14:19,  3.92it/s]
11094it [14:19,  3.91it/s]
11095it [14:20,  3.91it/s]
11096it [14:20,  3.91it/s]
11097it [14:20,  3.87it/s]
11098it [14:20,  3.79it/s]
11099it [14:21,  3.75it/s]
11100it [14:21,  3.71it/s]
11101it [14:21,  3.76it/s]
11102it [14:22,  3.78it/s]
11103it [14:22,  3.75it/s]
11104it [14:22,  3.71it/s]
11141it [14:22,  5.24it/s]
11146it [14:23,  6.71it/s]
11150it [14:23,  6.18it/s]
11153it [14:24,  5.27it/s]
11156it [14:25,  4.76it/s]
11158it [14:25,  4.43it/s]
11160it [14:26,  4.26it/s]
1

11760it [15:54,  3.64it/s]
11761it [15:55,  3.60it/s]
11762it [15:55,  3.62it/s]
11763it [15:55,  3.58it/s]
11764it [15:55,  3.59it/s]
11765it [15:56,  3.62it/s]
11766it [15:56,  3.63it/s]
11767it [15:56,  3.63it/s]
11768it [15:57,  3.55it/s]
11769it [15:57,  3.56it/s]
11770it [15:57,  3.58it/s]
11771it [15:57,  3.55it/s]
11772it [15:58,  3.51it/s]
11773it [15:58,  3.48it/s]
11774it [15:58,  3.53it/s]
11775it [15:59,  3.57it/s]
11776it [15:59,  3.56it/s]
11777it [15:59,  3.56it/s]
11778it [15:59,  3.54it/s]
11779it [16:00,  3.52it/s]
11780it [16:00,  3.49it/s]
11781it [16:00,  3.49it/s]
11782it [16:01,  3.54it/s]
11783it [16:01,  3.57it/s]
11784it [16:01,  3.60it/s]
11785it [16:01,  3.54it/s]
11786it [16:02,  3.57it/s]
11787it [16:02,  3.53it/s]
11788it [16:02,  3.49it/s]
11789it [16:03,  3.52it/s]
11790it [16:03,  3.54it/s]
11791it [16:03,  3.52it/s]
11792it [16:03,  3.57it/s]
11793it [16:04,  3.59it/s]
11794it [16:04,  3.61it/s]
11795it [16:04,  3.55it/s]
11796it [16:05,  3.58it/s]
1

12063it [17:21,  3.38it/s]
12064it [17:21,  3.41it/s]
12065it [17:22,  3.41it/s]
12066it [17:22,  3.43it/s]
12067it [17:22,  3.43it/s]
12068it [17:23,  3.44it/s]
12069it [17:23,  3.41it/s]
12070it [17:23,  3.39it/s]
12071it [17:23,  3.40it/s]
12072it [17:24,  3.42it/s]
12073it [17:24,  3.40it/s]
12074it [17:24,  3.35it/s]
12075it [17:25,  3.36it/s]
12076it [17:25,  3.39it/s]
12077it [17:25,  3.41it/s]
12078it [17:26,  3.43it/s]
12079it [17:26,  3.37it/s]
12080it [17:26,  3.40it/s]
12081it [17:26,  3.42it/s]
12082it [17:27,  3.43it/s]
12083it [17:27,  3.38it/s]
12084it [17:27,  3.34it/s]
12085it [17:28,  3.38it/s]
12086it [17:28,  3.40it/s]
12087it [17:28,  3.42it/s]
12088it [17:28,  3.33it/s]
12089it [17:29,  3.34it/s]
12090it [17:29,  3.36it/s]
12091it [17:29,  3.39it/s]
12092it [17:30,  3.38it/s]
12093it [17:30,  3.40it/s]
12094it [17:30,  3.41it/s]
12095it [17:31,  3.37it/s]
12096it [17:31,  3.34it/s]
12097it [17:31,  3.31it/s]
12098it [17:31,  3.28it/s]
12099it [17:32,  3.33it/s]
1

In [20]:
new_y.shape

(1001, 5)

In [ ]:
np.save('X_balanced', X)
np.save('y_balanced', y)

In [61]:
model = Sequential()

model.add(Conv2D(filters = 16, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (96,96,3)))
model.add(Conv2D(filters = 16, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.5))

model.add(Conv2D(filters = 16, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 16, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.5))


model.add(Flatten())
model.add(Dense(128, activation = "relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(4, activation = "softmax"))

In [62]:
optimizer = Adam(lr=0.0001)
model.compile(optimizer = optimizer, loss = "categorical_crossentropy")

In [64]:
model.fit(X, y, epochs=5, validation_split=0.2, batch_size=600, verbose=1, shuffle=True)

Train on 5753 samples, validate on 1439 samples
Epoch 1/5
5753/5753 [==============================] - ETA: 2s - loss: 0.448 - ETA: 2s - loss: 0.498 - ETA: 2s - loss: 0.515 - ETA: 1s - loss: 0.505 - ETA: 1s - loss: 0.506 - ETA: 1s - loss: 0.508 - ETA: 0s - loss: 0.491 - ETA: 0s - loss: 0.486 - ETA: 0s - loss: 0.490 - 3s 565us/step - loss: 0.4898 - val_loss: 0.5111
Epoch 2/5
5753/5753 [==============================] - ETA: 2s - loss: 0.397 - ETA: 2s - loss: 0.410 - ETA: 1s - loss: 0.439 - ETA: 1s - loss: 0.454 - ETA: 1s - loss: 0.460 - ETA: 1s - loss: 0.461 - ETA: 0s - loss: 0.467 - ETA: 0s - loss: 0.456 - ETA: 0s - loss: 0.458 - 3s 559us/step - loss: 0.4637 - val_loss: 0.5323
Epoch 3/5
5753/5753 [==============================] - ETA: 2s - loss: 0.427 - ETA: 2s - loss: 0.435 - ETA: 1s - loss: 0.445 - ETA: 1s - loss: 0.449 - ETA: 1s - loss: 0.448 - ETA: 1s - loss: 0.446 - ETA: 0s - loss: 0.447 - ETA: 0s - loss: 0.452 - ETA: 0s - loss: 0.456 - 3s 562us/step - loss: 0.4547 - val_loss: 0.

In [60]:
# Saving the Model
model.save('Models/first_modelv2_4out')

In [6]:
# Loading the Model
from keras.models import load_model
model = load_model('Models/first_modelv1_35_1_02_2_5')

In [67]:
#scores = []
#choices = []
env = CarRacing()
for each_game in range(2):
    #score = 0
    #game_memory = []
    prev_obs = env.reset()
    for i in range(1500):
        env.render()
        out = model.predict(prev_obs.reshape(-1,96,96,3))[0]
     
        output = np.argmax(out)
        action = np.zeros(4)
        np.put(action, output, 1)
        #choices.append(action)
        if np.array_equal(action, np.array([0,0,1,0])):
            action=np.array([0,1,0])
        elif np.array_equal(action, np.array([0,0,0,1])):
            action=np.array([0,0,0.8])
        elif np.array_equal(action, np.array([0,1,0,0])):
            action=np.array([1,0,0])
        elif np.array_equal(action, np.array([1,0,0,0])):
            action=np.array([-1,0,0])
            
        turn = out[1] if out[1]>out[0] else -out[0]
        turn = np.append(turn, out[2])
        turn = np.append(turn, out[3]*0.3)
        print(turn)
        new_observation, reward, done, info = env.step(turn)
        print(action, "by nn")
        prev_obs = new_observation
        #game_memory.append([new_observation, action])
        #score+=reward
        if done: 
            break
    #scores.append(score)
    #print('Episode {} Score: {}'.format(each_game+1,score))
    env.close()
#print('Average Score:',sum(scores)/len(scores))

Track generation: 1021..1288 -> 267-tiles track
[-0.69802141  0.16171849  0.01762932]
[-1  0  0] by nn
[0.0044453  0.54983193 0.13268716]
[0 1 0] by nn
[0.0037163  0.93275964 0.01880074]
[0 1 0] by nn
[0.46728367 0.51236016 0.00483471]
[0 1 0] by nn
[0.87479007 0.10800276 0.00102029]
[1 0 0] by nn
[0.46269229 0.50983006 0.00518329]
[0 1 0] by nn
[0.55491465 0.34977037 0.01372239]
[1 0 0] by nn
[-0.79753858  0.05719572  0.02134246]
[-1  0  0] by nn
[-0.91863316  0.01871419  0.00982082]
[-1  0  0] by nn
[-0.83636349  0.05686606  0.01904367]
[-1  0  0] by nn
[-0.8326152   0.05400105  0.02028446]
[-1  0  0] by nn
[-0.70508456  0.10344926  0.04340976]
[-1  0  0] by nn
[-0.68191445  0.13535279  0.03505745]
[-1  0  0] by nn
[-0.73616207  0.12031837  0.01808924]
[-1  0  0] by nn
[-0.74300945  0.0867701   0.01561001]
[-1  0  0] by nn
[-0.65653259  0.07477947  0.01310277]
[-1  0  0] by nn
[-0.65742499  0.04617097  0.00709467]
[-1  0  0] by nn
[-0.61228245  0.02959459  0.00579764]
[-1  0  0] by n

[-0.99179453  0.00227763  0.00149926]
[-1  0  0] by nn
[-0.98988944  0.00277997  0.00184408]
[-1  0  0] by nn
[-0.99012977  0.00248974  0.00188542]
[-1  0  0] by nn
[-0.98200041  0.00387251  0.00366594]
[-1  0  0] by nn
[-0.94627255  0.01360773  0.01051666]
[-1  0  0] by nn
[-0.90038222  0.01362742  0.02153527]
[-1  0  0] by nn
[-0.79991776  0.03657136  0.03921018]
[-1  0  0] by nn
[-0.70932233  0.05267298  0.05390107]
[-1  0  0] by nn
[-0.59231138  0.05502239  0.06378899]
[-1  0  0] by nn
[-0.54137832  0.02823734  0.07654905]
[-1  0  0] by nn
[-0.44721201  0.0216386   0.07278988]
[-1  0  0] by nn
[0.41740394 0.0159031  0.08367727]
[1 0 0] by nn
[0.42817011 0.01042589 0.08687615]
[1 0 0] by nn
[0.54214329 0.00745694 0.08276738]
[1 0 0] by nn
[0.44210717 0.00676709 0.1243039 ]
[1 0 0] by nn
[0.46354771 0.00516576 0.12340963]
[1 0 0] by nn
[0.39811677 0.0024216  0.14175246]
[0.  0.  0.8] by nn
[0.4226476  0.00221902 0.10524226]
[1 0 0] by nn
[0.31574771 0.00319307 0.16837762]
[0.  0.  0.

[-9.91804957e-01  5.04950527e-03  3.77595972e-04]
[-1  0  0] by nn
[-9.89497781e-01  6.92692818e-03  3.92093975e-04]
[-1  0  0] by nn
[-9.80091035e-01  1.49637917e-02  5.52772405e-04]
[-1  0  0] by nn
[-0.91116482  0.07647883  0.0015847 ]
[-1  0  0] by nn
[-0.62426215  0.32573786  0.00460325]
[-1  0  0] by nn
[0.19173923 0.62028617 0.00753243]
[0 1 0] by nn
[0.53358585 0.42046237 0.0045899 ]
[1 0 0] by nn
[0.83850133 0.14546493 0.00225   ]
[1 0 0] by nn
[0.96655631 0.02687627 0.00153661]
[1 0 0] by nn
[0.98688799 0.00694384 0.00170122]
[1 0 0] by nn
[0.99492455 0.00103312 0.00118294]
[1 0 0] by nn
[9.96100664e-01 3.41684092e-04 1.05830587e-03]
[1 0 0] by nn
[9.91592288e-01 3.10025818e-04 2.42582532e-03]
[1 0 0] by nn
[9.93924081e-01 1.60153708e-04 1.77274514e-03]
[1 0 0] by nn
[9.90035594e-01 1.70111409e-04 2.93697109e-03]
[1 0 0] by nn
[9.92284000e-01 1.68425089e-04 2.26301774e-03]
[1 0 0] by nn
[9.94476140e-01 1.72523141e-04 1.60351354e-03]
[1 0 0] by nn
[9.95641589e-01 2.11514693e-0

[0.96066594 0.02583396 0.00397179]
[1 0 0] by nn
[0.9718554  0.01751255 0.00314015]
[1 0 0] by nn
[0.97216356 0.01962106 0.00239755]
[1 0 0] by nn
[0.97212279 0.02018733 0.00223809]
[1 0 0] by nn
[0.96746325 0.02310701 0.00272895]
[1 0 0] by nn
[0.94314796 0.04800783 0.00255374]
[1 0 0] by nn
[0.89005071 0.09987907 0.00289858]
[1 0 0] by nn
[0.7311635  0.25637382 0.0035404 ]
[1 0 0] by nn
[0.40725961 0.57842821 0.00388092]
[0 1 0] by nn
[0.19202805 0.78484011 0.00493018]
[0 1 0] by nn
[0.13071662 0.82062215 0.00688424]
[0 1 0] by nn
[-0.19314083  0.69099665  0.00726544]
[0 1 0] by nn
[-0.64948523  0.27748075  0.00550688]
[-1  0  0] by nn
[-0.83992302  0.11843597  0.00435831]
[-1  0  0] by nn
[-0.89726436  0.0730603   0.00442123]
[-1  0  0] by nn
[-0.91144592  0.062049    0.00486513]
[-1  0  0] by nn
[-0.92042351  0.05245125  0.0062602 ]
[-1  0  0] by nn
[-0.94685513  0.03314928  0.0046367 ]
[-1  0  0] by nn
[-0.96380544  0.02182689  0.0029922 ]
[-1  0  0] by nn
[-0.96842682  0.01918411

[0.30867183 0.67944813 0.00273573]
[0 1 0] by nn
[0.15733279 0.82733762 0.00203837]
[0 1 0] by nn
[-0.08621     0.82660019  0.00300468]
[0 1 0] by nn
[-0.35507733  0.57542831  0.00309162]
[0 1 0] by nn
[-0.70843679  0.26089203  0.00259988]
[-1  0  0] by nn
[-0.87574297  0.10618211  0.00225478]
[-1  0  0] by nn
[-0.93917924  0.04789996  0.00250391]
[-1  0  0] by nn
[-0.94858712  0.03923823  0.0030185 ]
[-1  0  0] by nn
[-0.96276057  0.02627268  0.00255828]
[-1  0  0] by nn
[-0.96026486  0.02647222  0.00304817]
[-1  0  0] by nn
[-0.97082788  0.01940876  0.00198807]
[-1  0  0] by nn
[-0.95075905  0.03026274  0.00337378]
[-1  0  0] by nn
[-0.94766861  0.03359871  0.00387753]
[-1  0  0] by nn
[-0.95282233  0.03107292  0.00280721]
[-1  0  0] by nn
[-0.91167122  0.05621516  0.00504987]
[-1  0  0] by nn
[-0.8598699   0.09055569  0.00616449]
[-1  0  0] by nn
[-0.76881343  0.12143179  0.01041632]
[-1  0  0] by nn
[-0.62374282  0.17122002  0.00924142]
[-1  0  0] by nn
[-0.5397141   0.21167214  0.

...

[1 0 0] by nn
[9.98212218e-01 2.19527719e-05 5.29485801e-04]
[1 0 0] by nn
[9.96619105e-01 3.16103979e-05 1.00426369e-03]
[1 0 0] by nn
[9.96740162e-01 4.57113965e-05 9.63420072e-04]
[1 0 0] by nn
[9.95411098e-01 1.20536199e-04 1.33888707e-03]
[1 0 0] by nn
[9.96181369e-01 2.73293699e-04 1.05989159e-03]
[1 0 0] by nn
[0.99257225 0.00144685 0.00178359]
[1 0 0] by nn
[0.98195386 0.00962715 0.00247009]
[1 0 0] by nn
[0.9373734  0.05457596 0.00220714]
[1 0 0] by nn
[0.6979776  0.28867546 0.00349405]
[1 0 0] by nn
[0.28488144 0.69594997 0.0044888 ]
[0 1 0] by nn
[0.0667803  0.90014571 0.00446761]
[0 1 0] by nn
[-0.28500256  0.66720027  0.00547451]
[0 1 0] by nn
[-0.90238035  0.08480927  0.00143715]
[-1  0  0] by nn
[-9.80618656e-01  1.46711171e-02  5.81129396e-04]
[-1  0  0] by nn
[-9.90414202e-01  5.39571140e-03  6.14044676e-04]
[-1  0  0] by nn
[-9.91738379e-01  3.38220969e-03  9.16772895e-04]
[-1  0  0] by nn
[-0.99237692  0.00228871  0.00122157]
[-1  0  0] by nn
[-9.94528353e-01  1.4451

[-0.91967463  0.01597551  0.01389904]
[-1  0  0] by nn
[-0.90273887  0.02687874  0.01491012]
[-1  0  0] by nn
[-0.89205718  0.03368836  0.01571887]
[-1  0  0] by nn
[-0.91009915  0.03560835  0.01022364]
[-1  0  0] by nn
[-0.8934527   0.04863709  0.01054576]
[-1  0  0] by nn
[-0.79588252  0.1148592   0.01018125]
[-1  0  0] by nn
[-0.78583062  0.13537429  0.00868382]
[-1  0  0] by nn
[-0.78879005  0.1483333   0.00688316]
[-1  0  0] by nn
[-0.62191463  0.27609086  0.00574275]
[-1  0  0] by nn
[-0.3384091   0.51979941  0.00608023]
[0 1 0] by nn
[0.1228554  0.7915076  0.00561534]
[0 1 0] by nn
[0.13697805 0.83746326 0.00395689]
[0 1 0] by nn
[0.21678673 0.76624656 0.00368363]
[0 1 0] by nn
[0.39810839 0.58816606 0.00370511]
[0 1 0] by nn
[0.71738559 0.2686516  0.0036812 ]
[1 0 0] by nn
[0.86589724 0.1219814  0.00337851]
[1 0 0] by nn
[0.92473984 0.06177317 0.00386603]
[1 0 0] by nn
[0.98333079 0.01080562 0.00164684]
[1 0 0] by nn
[0.98864436 0.00601392 0.00154834]
[1 0 0] by nn
[0.98327637 

[0.97373474 0.00570122 0.00613384]
[1 0 0] by nn
[0.97994012 0.00454952 0.00462276]
[1 0 0] by nn
[0.97485965 0.00976074 0.00452161]
[1 0 0] by nn
[0.95446068 0.02969162 0.00452641]
[1 0 0] by nn
[0.92742926 0.05870724 0.00387541]
[1 0 0] by nn
[0.69175655 0.2860969  0.00573639]
[1 0 0] by nn
[0.3776179  0.5995959  0.00591571]
[0 1 0] by nn
[0.07983145 0.90309244 0.00363681]
[0 1 0] by nn
[-0.05165243  0.90587032  0.00404608]
[0 1 0] by nn
[-0.59556127  0.37129518  0.00358401]
[-1  0  0] by nn
[-0.94037652  0.04976882  0.00112583]
[-1  0  0] by nn
[-9.76081252e-01  1.69441402e-02  9.14863637e-04]
[-1  0  0] by nn
[-0.98526841  0.00880932  0.00103755]
[-1  0  0] by nn
[-0.98801708  0.00583625  0.00141951]
[-1  0  0] by nn
[-0.99087942  0.00391924  0.00110619]
[-1  0  0] by nn
[-0.99075341  0.00335476  0.0012586 ]
[-1  0  0] by nn
[-9.94640708e-01  2.11997842e-03  6.20980561e-04]
[-1  0  0] by nn
[-9.94977415e-01  2.14316859e-03  4.46572015e-04]
[-1  0  0] by nn
[-9.95459199e-01  1.92919

[0.90655535 0.07963797 0.00383702]
[1 0 0] by nn
[0.8016057  0.17893    0.00436094]
[1 0 0] by nn
[0.5667665  0.39900112 0.00487827]
[1 0 0] by nn
[0.32849151 0.60366237 0.00431132]
[0 1 0] by nn
[-0.17176457  0.70427918  0.00524748]
[0 1 0] by nn
[-0.51323903  0.45085722  0.00390533]
[-1  0  0] by nn
[-0.81735849  0.16346839  0.00275358]
[-1  0  0] by nn
[-0.95783031  0.03392327  0.00127406]
[-1  0  0] by nn
[-9.84122396e-01  9.67406947e-03  9.26846056e-04]
[-1  0  0] by nn
[-9.91352558e-01  3.79622681e-03  7.49003515e-04]
[-1  0  0] by nn
[-0.99062538  0.00394146  0.00115396]
[-1  0  0] by nn
[-9.93798077e-01  2.52778106e-03  6.75470475e-04]
[-1  0  0] by nn
[-9.92593944e-01  3.24246427e-03  7.54266581e-04]
[-1  0  0] by nn
[-9.92571414e-01  4.25158860e-03  5.16963028e-04]
[-1  0  0] by nn
[-0.97555965  0.01681368  0.00140775]
[-1  0  0] by nn
[-0.9295488   0.05835789  0.0020567 ]
[-1  0  0] by nn
[-0.78321707  0.19377087  0.00404681]
[-1  0  0] by nn
[-0.55013525  0.40391698  0.0048

[1 0 0] by nn
[0.96657693 0.02598982 0.00221878]
[1 0 0] by nn
[0.97355032 0.0182531  0.00245001]
[1 0 0] by nn
[0.98401099 0.00922471 0.00202441]
[1 0 0] by nn
[0.98864239 0.00657422 0.00142856]
[1 0 0] by nn
[0.9802326  0.01334436 0.00190232]
[1 0 0] by nn
[0.97268957 0.0203567  0.00205254]
[1 0 0] by nn
[0.93456483 0.05665519 0.00249973]
[1 0 0] by nn
[0.81715232 0.17015661 0.00316716]
[1 0 0] by nn
[0.61957026 0.358632   0.00440889]
[1 0 0] by nn
[0.38265637 0.57754254 0.00432922]
[0 1 0] by nn
[0.21577802 0.68271279 0.00556844]
[0 1 0] by nn
[-0.2807475   0.64251965  0.00562617]
[0 1 0] by nn
[-0.62473935  0.34124744  0.00449399]
[-1  0  0] by nn
[-0.80259168  0.17522177  0.00348683]
[-1  0  0] by nn
[-0.91804606  0.06656644  0.00235751]
[-1  0  0] by nn
[-0.96421999  0.02461819  0.00195496]
[-1  0  0] by nn
[-0.96553838  0.02388202  0.00223521]
[-1  0  0] by nn
[-0.96858561  0.02177789  0.00216631]
[-1  0  0] by nn
[-0.95138347  0.03388235  0.00376646]
[-1  0  0] by nn
[-0.943633